# How to combine MGnify Run data with pHmmer sequence search results in a TSV 

User request:
`After using the sequence search tool, I tried to download the results but the information about the RUNs is lost in the .csv file. How can I include it?`

The [MGnify Run](https://emg-docs.readthedocs.io/en/latest/glossary.html?highlight=runs#term-Run) information for a search result hit is available in the JSON download data, but not in the TSV file. To augment the TSV with that data from the JSON, we can manipulate the JSON slightly into a table format, and join the two.

In [1]:
import pandas as pd
import json

Files downloaded from a Sequence Search result, e.g. https://www.ebi.ac.uk/metagenomics/sequence-search/download/F5AB62C8-19F9-11EC-BD62-FC88FDD569CA/score

In [2]:
TSV_FILE = '4C6B46DA-19F2-11EC-BD62-FC88FDD569CA.1.csv'
JSON_FILE = '4C6B46DA-19F2-11EC-BD62-FC88FDD569CA.1.json'

In [3]:
results_table = pd.read_csv(TSV_FILE, sep='\t')
results_table.set_index('Target Name', inplace=True)

Now load the JSON as a table, and extract the Run accessions.

In [4]:
with open(JSON_FILE, 'r') as json_file:
    hits_list = json.load(json_file).get('results').get('hits')

json_results = pd.json_normalize(hits_list)
json_results.rename(columns={'name': 'Target Name'}, inplace=True)
json_results['Runs'] = json_results.apply(lambda hit: [run[0] for run in hit['mgnify.runs'] or []], axis=1)
json_results = json_results[['Target Name', 'Runs']]
json_results.set_index('Target Name', inplace=True)

In [5]:
results_table = results_table.join(json_results, on=['Target Name'], how='left')
results_table.to_csv('phmmer_result_with_runs.tsv', sep='\t')
results_table

,Target Accession,Target Length,Query Name,Query Accession,Query Length,E-value,Score,Bias,Domain Index,Domain Count,...,Query Ali. End,Target Ali. Start,Target Ali. End,Target Env. Start,Target Env. End,Acc,Description,Mapped PDB(s),Number of Identical Sequences,Runs
Target Name,,,,,,,,,,,,,,,,,,,,,
MGYP000006146753,NaN,409,>tr|F2BTU6|F2BTU6_STRSA Arginine deiminase OS=...,-,409,3.500000e-265,890.8,4.0,1,1,...,409,1,409,1,409,1.00,MGYP000006146753,NaN,0,[]
MGYP000160620212,NaN,409,>tr|F2BTU6|F2BTU6_STRSA Arginine deiminase OS=...,-,409,2.000000e-255,858.6,3.9,1,1,...,409,1,409,1,409,1.00,MGYP000160620212,NaN,0,[ERR1474574]
MGYP000016395208,NaN,409,>tr|F2BTU6|F2BTU6_STRSA Arginine deiminase OS=...,-,409,1.100000e-234,790.3,3.2,1,1,...,409,1,409,1,409,1.00,MGYP000016395208,NaN,0,[SRR6231191]
MGYP000015122295,NaN,410,>tr|F2BTU6|F2BTU6_STRSA Arginine deiminase OS=...,-,409,3.400000e-233,785.4,3.3,1,1,...,409,3,410,2,410,1.00,MGYP000015122295,NaN,0,"[ERR526022, SRR3132381, SRR3132143, SRR3132237..."
MGYP000072312189,NaN,410,>tr|F2BTU6|F2BTU6_STRSA Arginine deiminase OS=...,-,409,6.700000e-233,784.5,3.4,1,1,...,409,5,410,1,410,0.99,MGYP000072312189,NaN,0,[ERR1135178]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MGYP000078628186,NaN,61,>tr|F2BTU6|F2BTU6_STRSA Arginine deiminase OS=...,-,409,9.100000e-03,26.8,0.0,1,1,...,51,6,50,1,59,0.87,MGYP000078628186,NaN,0,[ERR2168994]
MGYP000197530100,NaN,312,>tr|F2BTU6|F2BTU6_STRSA Arginine deiminase OS=...,-,409,9.300000e-03,26.8,0.0,1,2,...,284,140,217,134,224,0.80,MGYP000197530100,NaN,0,"[SRR1628698, SRR1628697]"
MGYP000197530100,NaN,312,>tr|F2BTU6|F2BTU6_STRSA Arginine deiminase OS=...,-,409,9.300000e-03,26.8,0.0,2,2,...,403,217,306,203,311,0.85,MGYP000197530100,NaN,0,"[SRR1628698, SRR1628697]"
